## Setup

In [1]:
import pandas as pd
import json

from tests.eas.load_tracking import FreqInvarianceTest
from trace import Trace
from trappy.plotter import plot_trace
from trappy.stats.grammar import Parser
from trappy import ILinePlot

In [2]:
import logging
from conf import LisaLogging
LisaLogging.setup()

2016-11-29 15:39:35,317 INFO    : root         : Using LISA logging configuration:
2016-11-29 15:39:35,318 INFO    : root         :   /home/brejac01/sources/lisa/logging.conf


## Run test workload

In [3]:
t = FreqInvarianceTest(methodName='test_task_util')

In [4]:
t.setUpClass()

2016-11-29 15:39:35,334 INFO    : LisaTest     : Setup tests execution engine...
2016-11-29 15:39:35,335 INFO    : TestEnv      : Using base path: /home/brejac01/sources/lisa
2016-11-29 15:39:35,335 INFO    : TestEnv      : Loading default (file) target configuration
2016-11-29 15:39:35,336 INFO    : TestEnv      : Loading target configuration [/home/brejac01/sources/lisa/target.config]...
2016-11-29 15:39:35,339 INFO    : TestEnv      : Loading custom (inline) test configuration
2016-11-29 15:39:35,340 INFO    : TestEnv      : Devlib modules to load: ['bl', u'cpuidle', 'cpufreq']
2016-11-29 15:39:35,341 INFO    : TestEnv      : Connecting linux target:
2016-11-29 15:39:35,341 INFO    : TestEnv      :   username : brendan
2016-11-29 15:39:35,342 INFO    : TestEnv      :       host : 192.168.2.2
2016-11-29 15:39:35,343 INFO    : TestEnv      :   password : password
2016-11-29 15:39:35,344 INFO    : TestEnv      : Connection settings:
2016-11-29 15:39:35,345 INFO    : TestEnv      :    {

## Examine trace from workload execution

The log `setUpClass` call above shows the experiments that were run. Use the function below to get a plot from a given experiment of the `util_avg` scheduler signal for that workload's tasks, along with the expected mean value.

In [8]:
def examine_experiment(idx, signal):
    experiment = t.executor.experiments[idx]
    
    print "Experiment configuration:" 
    print json.dumps(experiment.conf, indent=4)
    events = t.te.test_conf["ftrace"]["events"]
    
    # todo add get_trace method
    tasks = experiment.wload.tasks.keys()
    #trace = Trace(t.te.platform, experiment.out_dir, events, tasks)
    #print "Trace plot:"
    #plot_trace(trace.ftrace)
    
    # Get observed signal
    signal_df = t.get_sched_signal(experiment, signal)
#    return signal_df
    # Get expected average value for util_avg signal
    expected_util_avg_mean = t.get_expected_util_avg(experiment)
    
    # Plot util avg signal with expected mean value
    util_avg_mean = pd.Series([expected_util_avg_mean], name="expected_util_avg", index=[signal_df.index[0]])
    df = pd.concat([signal_df, util_avg_mean], axis=1).fillna(method="ffill")
    ILinePlot(df, column=[signal, "expected_util_avg"], drawstyle="steps-post").view()

In [10]:
df = examine_experiment(0, "load_avg")

2016-11-29 15:41:15,565 INFO    : Trace        : Parsing FTrace format...


Experiment configuration:
{
    "dtb": null, 
    "kernel": null, 
    "tag": "freq_450000", 
    "flags": "ftrace", 
    "cpufreq": {
        "freqs": {
            "1": 450000
        }, 
        "governor": "userspace"
    }
}


2016-11-29 15:41:16,028 INFO    : Trace        : Collected events spans a 9.498 [s] time interval
2016-11-29 15:41:16,029 INFO    : Trace        : Set plots time range to (0.000000, 9.497775)[s]
2016-11-29 15:41:16,030 INFO    : Analysis     : Registering trace analysis modules:
2016-11-29 15:41:16,030 INFO    : Analysis     :    tasks
2016-11-29 15:41:16,031 INFO    : Analysis     :    eas
2016-11-29 15:41:16,032 INFO    : Analysis     :    latency
2016-11-29 15:41:16,033 INFO    : Analysis     :    frequency
2016-11-29 15:41:16,034 INFO    : Analysis     :    functions
2016-11-29 15:41:16,034 INFO    : Analysis     :    idle
2016-11-29 15:41:16,035 INFO    : Analysis     :    status
2016-11-29 15:41:16,036 INFO    : Analysis     :    cpus


""
""
